세종말뭉치의 품사정보는 형태소분석을 하기 위한 자료이며, 데이터분석의 수준에서는 일반명사/고유명사 등을 분리하거나 목적격/주격 조사를 구분할 필요가 없기 때문에 품사를 단순화 하여 볼 필요가 있다. 또한 비지도학습 기반 토크나이저인 soynlp tokenizers를 위해서도 품사는 합쳐질 필요가 있다. 아래는 '진짜했다니까' 라는 띄어쓰기 오류가 있는 짧은 문장에 대하여 우리가 기대하는 결과이다. '했다니까'는 '하다/동사'가 활용이 된 형태이며, 한국어 품사에 '어미(Eomi)'는 존재하지 않는다. Suffix인 '어미'는 형태소이다. 형태소분석과 품사판별을 나눠서 풀어보자

    Part of speech tagging: 진짜/명사 + 했다니까/동사
    Morphological analysis: 진짜/명사 + 하/동사 + 았/선어말어미 + 다니까/어말어미
    POS tagging + Lemmatization: 진짜/명사 + 하다/동사
    L/R POS tagging:  진짜/명사 + 했/동사R + 다니까/어미E

특히 용언이 활용되었을 때가 까다로운데, lemmatization을 하지 않는 품사판별이나 L/R pos tagging에서는 surfacial form 그대로 단어를 인식한다. 단, 명사의 경우 -ㄴ, -ㄹ은 hard rules로 분리한다. 

    진짠데 = 진짜 + ㄴ데
    
아래는 품사판별을 위한 사전 형성이다. 품사 판별은 단어의 surfacial form만 보고도 맞출 수 있어야 한다. 토크나이징이 된 단어열 (word sequence)에 대하여 tagging이 되면 된다. Surfacial form을 인식하면 되기 때문에 character-level로 작업을 할 수 있다. Word segmentation + Tagging이 되면 품사 판별이 가능하다. 

## 품사 판별 (Part of Speech Tagging)을 위한 사전 구축

In [21]:
with open('../../sejong/data/processed/lr/tokentable_simple.txt', encoding='utf-8') as f:
    for _ in range(10):
        print(next(f).strip())

그	그/MM	그/Determiner	80726
수	수/NNB	수/Noun	67991
이	이/MM	이/Determiner	45777
것이다.	것/NNB+이/VCP+다/EF+./SF	것/Noun+이/Adjective+다/Eomi	37808
있다.	있/VX+다/EF+./SF	있/Verb+다/Eomi	30548
한	한/MM	한/Determiner	30536
있는	있/VX+는/ETM	있/Verb+는/Eomi	28946
있다.	있/VV+다/EF+./SF	있/Verb+다/Eomi	26195
있는	있/VV+는/ETM	있/Verb+는/Eomi	25045
것은	것/NNB+은/JX	것/Noun+은/Josa	24816


In [32]:
'abc'.split()[3:]

[]

In [76]:
from soynlp.hangle import normalize

def as_lr(text, morphomes):
    text = normalize(text, english=True, number=True)
    
    if morphomes[0][1] == 'Adverb':
        adverb_end_index = max([idx for idx, (_, _) in filter(lambda x:x[1][1] == 'Adverb', enumerate(morphomes))]) + 1
        morphomes = [(''.join([w for w,_ in morphomes[:adverb_end_index]]), 'Adverb')] + morphomes[adverb_end_index:]
    if morphomes[0][1] == 'Adverb' and len(morphomes) >= 2 and (morphomes[1][1] == 'Verb' or morphomes[1][1] == 'Adjective'):
        morphomes = [(morphomes[0][0]+morphomes[1][0], morphomes[1][1])] + morphomes[2:]
    if morphomes[0][1] == 'Exclamation':
        all_Exclamation = True
        for _, tag in morphomes:
            if tag != 'Exclamation':
                all_Exclamation = False
        if all_Exclamation:
            morphomes = [(text, 'Exclamation')]
    
    if len(morphomes) == 1:
        return morphomes
    
    if morphomes[0][1] == 'Noun':
        noun_end_index = max([idx for idx, (_, _) in filter(lambda x:x[1][1] == 'Noun', enumerate(morphomes))]) + 1
        L = ''.join([w for w, _ in morphomes[:noun_end_index]])
        if noun_end_index == len(morphomes):
            return [(L, 'Noun')]
        R = text[len(L):]
        return [(L, 'Noun'), (R, morphomes[noun_end_index][1], morphomes[noun_end_index:])]
    if (morphomes[0][1] == 'Verb' or morphomes[0][1] == 'Adjective') and (morphomes[-1][1] == 'Eomi', morphomes[-1][1] == 'Josa'):
        return [(text, morphomes[0][1], morphomes)]
    if (morphomes[0][1] == 'Adverb') and (morphomes[-1][1] == 'Noun'):
        return [(text, 'Noun')]
    if (morphomes[0][1] == 'Adverb') and (morphomes[-1][1] == 'Josa' or morphomes[-1][1] == 'Eomi'):
        return [(text, 'Adverb')]
    if (morphomes[0][1] == 'Adverb' or morphomes[0][1] == 'Determiner') and (morphomes[-1][1] == 'Determiner'):
        return [(text, 'Determiner')]
    if (morphomes[0][1] == 'Exclamation') and (morphomes[-1][1] == 'Josa' or morphomes[-1][1] == 'Eomi'):
        return [(text, 'Exclamation')]
    if morphomes[-1][1] == 'Noun':
        return [(text, 'Noun')]
#     if 
        

In [88]:
from collections import defaultdict
stem_table = defaultdict(lambda: defaultdict(lambda: 0))

with open('../../sejong/data/processed/lr/tokentable_simple.txt', encoding='utf-8') as fi:
    with open('../../sejong/data/processed/lr/tokentable_simple_lr.txt', 'w', encoding='utf-8') as fo:
        for line in fi:
            text, row, pos, count = line.strip().split('\t')
            pos_ = [m.split('/') for m in pos.split('+')]
            lr = as_lr(text, pos_)
            if lr:
                if len(lr[-1]) == 3:
                    R = '{}/{}'.format(lr[-1][0], lr[-1][1])
                    stem = tuple([tuple(m) for m in lr[-1][2]])
                    stem_table[R][stem] += int(count.strip())
                    lr[-1] = lr[-1][:2]
                lr_ = '+'.join(['{}/{}'.format(w,t) for w,t in lr])
                fo.write('{}\t{}\t{}\t{}\t{}\n'.format(text, row, pos, lr_, count.strip()))
#             else:
#                 print('{}\t{}\t\t{}\n{}\n'.format(text, morphomes, count.strip(), lr))
        

In [208]:
from soynlp.hangle import decompose, compose

def complify_jamo(s):
    last = s[-1][0]
    if len(last) >= 2:
        # ㅏㅆ다 -> 았다
        if (12623 <= ord(last[0]) <= 12643) and (12593 <= ord(last[1]) <= 12622):
            s = tuple(list(s[:-1]) + [(compose('ㅇ', last[0], last[1]) + last[2:], s[-1][1])])
    return s

def hangle_normalizer(w):
    replacer = {
        'ᆼ':'ㅇ',
        'ᄆ':'ㅁ',
        'ᄇ':'ㅂ',
        'ᆯ':'ㄹ',
        'ᆫ':'ㄴ'
    }
    return ''.join([replacer.get(c, c) for c in w])

def clean_stem_table(stem_table):    
    clean_table = {}
    for R, stems in sorted(stem_table.items(), key=lambda x:-sum(x[1].values())):
        R = R.strip()
        R = normalize(R.split('/')[0]) + '/' + R.split('/')[1]
        if (not R) or (R[0] == '/') or (R in clean_table) or (' ' in R):
            continue
        freq = sum(stems.values())
#         if freq <= 1:
#             continue
            
        if len(stems) == 1:
            best = list(stems.keys())[0]
        
        best = sorted(stems.items(), key=lambda x:-x[1])[0][0]
        best = complify_jamo(best)
        best = [(hangle_normalizer(w), t) for w,t in best]
        if (len(best) == 1) and (best[0][1] == 'Josa' or best[0][1] == 'Eomi') and (R.split('/')[0] != best[0][0]):
            continue
        clean_table[R] = ('+'.join(['{}/{}'.format(m, tag) for m,tag in best]), freq)
    return clean_table

In [209]:
clean_table = clean_stem_table(stem_table)
len(clean_table)

176064

In [210]:
for R, stem in sorted(clean_table.items(), key=lambda x:x[1][1])[100:130]:
    print('%s --> %s'%(R, stem))

둘러댔는데/Verb --> ('둘러대/Verb+었는데/Eomi', 1)
살겠다니/Verb --> ('살/Verb+겠다니/Eomi', 1)
그렇구말구요/Adjective --> ('그렇/Adjective+구말구요/Eomi', 1)
벗어달라고/Verb --> ('벗/Verb+어달라고/Eomi', 1)
일으키려다가/Verb --> ('일으키/Verb+려다가/Eomi', 1)
불으락푸르락했다/Verb --> ('불그락푸르락하/Verb+았다/Eomi', 1)
많으시리라/Adjective --> ('많/Adjective+으시리라/Eomi', 1)
나쁘구/Adjective --> ('나쁘/Adjective+구/Eomi', 1)
거치다가는/Verb --> ('거치/Verb+다가는/Eomi', 1)
더웠다가/Adjective --> ('덥/Adjective+었다가/Eomi', 1)
부럴/Adjective --> ('부럽/Adjective+ㄹ/Eomi', 1)
우러나오게/Verb --> ('우러나오/Verb+게/Eomi', 1)
낳는/Adjective --> ('낳/Adjective+는/Eomi', 1)
밝혀내자/Verb --> ('밝혀내/Verb+자/Eomi', 1)
밝혀놓았으므로/Verb --> ('밝히/Verb+어놓았으므로/Eomi', 1)
해먹습니다/Verb --> ('하/Verb+아먹습니다/Eomi', 1)
줏었다/Verb --> ('줏/Verb+었다/Eomi', 1)
지낸다고만/Verb --> ('지내/Verb+ㄴ다고만/Eomi', 1)
짓이겼지만/Verb --> ('짓이기/Verb+었지만/Eomi', 1)
파헤치겠다는/Verb --> ('파헤치/Verb+겠다는/Eomi', 1)
더해가며/Verb --> ('더하/Verb+아가며/Eomi', 1)
흥치며/Verb --> ('흥치/Verb+며/Eomi', 1)
게에/Josa --> ('게에/Josa', 1)
잘해봤자/Verb --> ('잘하/Verb+아보았자/Eomi', 1)
맡겼단다/Verb --> ('맡기/Verb+

In [211]:
for R, stem in sorted(clean_table.items(), key=lambda x:-x[1][1])[100:130]:
    print('%s --> %s'%(R, stem))

않은/Verb --> ('않/Verb+은/Eomi', 6135)
좋은/Adjective --> ('좋/Adjective+은/Eomi', 6105)
밖에/Josa --> ('밖에/Josa', 5901)
있어/Verb --> ('있/Verb+어/Eomi', 5823)
했던/Verb --> ('하/Verb+았던/Eomi', 5763)
일/Adjective --> ('이/Adjective+ㄹ/Eomi', 5748)
되지/Verb --> ('되/Verb+지/Eomi', 5670)
지난/Verb --> ('지나/Verb+ㄴ/Eomi', 5667)
마다/Josa --> ('마다/Josa', 5381)
들어/Verb --> ('들/Verb+어/Eomi', 5369)
해도/Verb --> ('하/Verb+아도/Eomi', 5353)
이라/Adjective --> ('이/Adjective+라/Eomi', 5311)
돼/Verb --> ('되/Verb+어/Eomi', 5150)
인데/Adjective --> ('이/Adjective+ㄴ데/Eomi', 5113)
으로는/Josa --> ('으로는/Josa', 4960)
크게/Adjective --> ('크/Adjective+게/Eomi', 4660)
있다고/Verb --> ('있/Verb+다고/Eomi', 4566)
있던/Verb --> ('있/Verb+던/Eomi', 4525)
알/Verb --> ('알/Verb+ㄹ/Eomi', 4458)
않을/Verb --> ('않/Verb+을/Eomi', 4396)
알고/Verb --> ('알/Verb+고/Eomi', 4388)
온/Verb --> ('오/Verb+ㄴ/Eomi', 4274)
있는데/Verb --> ('있/Verb+는데/Eomi', 4132)
한테/Josa --> ('한테/Josa', 4131)
보다는/Josa --> ('보다는/Josa', 4098)
않는다/Verb --> ('않/Verb+는다/Eomi', 4049)
주는/Verb --> ('주/Verb+는/Eomi', 4001

#### to DataFrame

In [212]:
data = [
    {
        'Word': R.split('/')[0],
        'Tag': R.split('/')[1],
        'Stem': stem,
        'Frequency': freq
    }
    for R, (stem, freq) in sorted(clean_table.items(), key=lambda x:-x[1][1])
]

import pandas as pd
stem_table_df = pd.DataFrame(data, columns=['Word', 'Tag', 'Stem', 'Frequency'])
stem_table_df.to_csv('../data/processed/lr/stem_table.csv', encoding='utf-8')

In [213]:
stem_table_df

,Word,Tag,Stem,Frequency
0,을,Josa,을/Josa,495674
1,의,Josa,의/Josa,464361
2,이,Josa,이/Josa,379235
3,에,Josa,에/Josa,340265
4,를,Josa,를/Josa,281138
5,가,Josa,가/Josa,249587
6,은,Josa,은/Josa,245650
7,는,Josa,는/Josa,214588
8,으로,Josa,으로/Josa,139157
9,도,Josa,도/Josa,114151


In [91]:
for R, stems in sorted(stem_table.items(), key=lambda x:-sum(x[1].values()))[:100]:
    print('\n%s'%R)
    for stem, count in sorted(stems.items(), key=lambda x:-x[1]):
        print('  > %s (%d)' % (stem, count))


을/Josa
  > (('을', 'Josa'),) (495523)
  > (('에서을', 'Josa'),) (88)
  > (('ᆯ', 'Josa'),) (19)
  > (('를', 'Josa'),) (15)
  > (('만을', 'Josa'),) (13)
  > (('이을', 'Josa'),) (13)
  > (('과을', 'Josa'),) (3)

의/Josa
  > (('의', 'Josa'),) (464320)
  > (('에서의', 'Josa'),) (22)
  > (('이의', 'Josa'),) (9)
  > (('과의', 'Josa'),) (8)
  > (('을', 'Josa'),) (1)
  > (('는의', 'Josa'),) (1)

이/Josa
  > (('이', 'Josa'),) (379141)
  > (('의', 'Josa'),) (34)
  > (('에서이', 'Josa'),) (28)
  > (('만이', 'Josa'),) (17)
  > (('이이', 'Josa'),) (11)
  > (('같이', 'Josa'),) (3)
  > (('과이', 'Josa'),) (1)

에/Josa
  > (('에', 'Josa'),) (340218)
  > (('에서에', 'Josa'),) (17)
  > (('의', 'Josa'),) (12)
  > (('이에', 'Josa'),) (6)
  > (('에에', 'Josa'),) (3)
  > (('에서', 'Josa'),) (2)
  > (('이', 'Josa'),) (2)
  > (('는에', 'Josa'),) (2)
  > (('밖에', 'Josa'),) (1)
  > (('도', 'Josa'),) (1)
  > (('의집에', 'Josa'),) (1)

를/Josa
  > (('를', 'Josa'),) (281127)
  > (('을', 'Josa'),) (10)
  > (('에', 'Josa'),) (1)

가/Josa
  > (('가', 'Josa'),) (249582)
  > (('ᆫ가